# 3번
MyDense class 구현.  
MyDense는 tensorflow.keras.layers.Dense와 똑같이 사용할 수 있다.  
MyDense의 생성자 매개 변수는 units, activation=None, use_bias = True, kernel_initializer = &#039;glorot_uniform&#039;, bias_initializer = &#039;zeros&#039; 이다  
MyDense에서 tensorflow.keras.layers.Conv1D, tensorflow.keras.layers.Flatten, tf.expand_dims를 사용해서 구현 가능.  
다시 이야기해 Conv1D층을 Dense층과 똑 같이 실행되게 사용할 수 있다는 것을 보여라.

In [1]:
from tensorflow.keras.layers import Flatten, Conv1D, Conv2D
from tensorflow.keras import Model
import tensorflow as tf

import matplotlib.pyplot as pit

In [2]:
class MyDense(tf.keras.layers.Layer):
    def __init__(self, units, activation=None, use_bias=True, kernel_initializer="glorot_uniform", bias_initializer="zeros"):
        super(MyDense, self).__init__()
        self.units = units
        self.activation = activation
        self.use_bias = use_bias
        self.kernel_initializer = kernel_initializer
        self.bias_initializer = bias_initializer
    
        self.flatten = Flatten()
        self.expand_dims = tf.expand_dims
        self.conv1d = Conv1D(units, 
                             1,
                             padding='same',
                             activation=self.activation, 
                             use_bias=self.use_bias,
                             kernel_initializer=self.kernel_initializer,
                             bias_initializer=self.bias_initializer)

    def call(self, input_tensor):
        x = self.flatten(input_tensor)
        x = self.expand_dims(x, axis=1)
        x = self.conv1d(x)
        output = self.flatten(x)
        return output

In [3]:
mnist = tf.keras.datasets.mnist
mnist

<module 'keras.api._v2.keras.datasets.mnist' from 'C:\\Users\\jyb\\Anaconda3\\envs\\private\\lib\\site-packages\\keras\\api\\_v2\\keras\\datasets\\mnist\\__init__.py'>

In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [5]:
print("Train shape", x_train.shape, y_train.shape)
print("Test shape", x_test.shape, y_test.shape)

Train shape (60000, 28, 28) (60000,)
Test shape (10000, 28, 28) (10000,)


In [6]:
print("Label shape", y_train.shape, y_test.shape)

Label shape (60000,) (10000,)


In [7]:
reshaped_train_x = x_train.reshape(-1, 28, 28, 1)
reshaped_test_x = x_test.reshape(-1, 28, 28, 1)

In [8]:
print("Reshaped shape", reshaped_train_x.shape, reshaped_test_x.shape)

Reshaped shape (60000, 28, 28, 1) (10000, 28, 28, 1)


In [9]:
## Nomralize
reshaped_train_x, reshaped_test_x = reshaped_train_x / 255.0, reshaped_test_x / 255.0

In [10]:
## Define  Model
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    MyDense(64),
    MyDense(10, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 128)               100480    
                                                                 
 my_dense (MyDense)          (None, 10)                2570      
                                                                 
Total params: 103,050
Trainable params: 103,050
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [12]:
epochs = 10
batch_size = 256
learning_rate = 0.01
verbose = 1
validation_split = 0.2

In [13]:
model.fit(reshaped_train_x, y_train,
          batch_size=batch_size, 
          epochs=epochs,
          verbose=verbose, 
          validation_split=validation_split)

Epoch 1/10
188/188 [==============================] - 1s 2ms/step - loss: 2.6240 - accuracy: 0.2205 - val_loss: 2.3022 - val_accuracy: 0.2038
Epoch 2/10
188/188 [==============================] - 0s 2ms/step - loss: 2.3025 - accuracy: 0.1972 - val_loss: 2.3022 - val_accuracy: 0.2038
Epoch 3/10
188/188 [==============================] - 0s 2ms/step - loss: 2.3025 - accuracy: 0.1972 - val_loss: 2.3022 - val_accuracy: 0.2038
Epoch 4/10
188/188 [==============================] - 0s 2ms/step - loss: 2.3025 - accuracy: 0.1972 - val_loss: 2.3022 - val_accuracy: 0.2038
Epoch 5/10
188/188 [==============================] - 0s 2ms/step - loss: 2.3025 - accuracy: 0.1972 - val_loss: 2.3022 - val_accuracy: 0.2038
Epoch 6/10
188/188 [==============================] - 0s 2ms/step - loss: 2.3025 - accuracy: 0.1972 - val_loss: 2.3022 - val_accuracy: 0.2038
Epoch 7/10
188/188 [==============================] - 0s 2ms/step - loss: 2.3025 - accuracy: 0.1972 - val_loss: 2.3022 - val_accuracy: 0.2038
Epoch 

In [14]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy: ', test_acc)

313/313 [==============================] - 0s 645us/step - loss: 2.3026 - accuracy: 0.1879
Test accuracy:  0.18790000677108765
